# Pre 

In [1]:
import math
import pdb
import random
import time
import pandas as pd
import pdb
import numpy as np
import itertools
import os
from skmultilearn.model_selection import IterativeStratification
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from skmultilearn.ensemble import RakelD
from skmultilearn.adapt import MLkNN
from skmultilearn.adapt import BRkNNbClassifier
from sklearn.neighbors import NearestNeighbors
from collections import Counter
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import euclidean
from sklearn.neighbors import NearestNeighbors
from skmultilearn.dataset import load_dataset
from skmultilearn.dataset import load_from_arff
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import GaussianNB
from skmultilearn.ensemble import RakelO
from skmultilearn.adapt import MLTSVM
from skmultilearn.adapt import MLARAM
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from enum import Enum
import sklearn.metrics as metrics
from scipy import sparse
from skmultilearn.model_selection import IterativeStratification
from skmultilearn.adapt import MLkNN
from skmultilearn.dataset import load_dataset
from sklearn.neighbors import NearestNeighbors
import sklearn.metrics as metrics
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from sklearn.tree import DecisionTreeClassifier
from skmultilearn.dataset import load_from_arff
from skmultilearn.ensemble import RakelD
from xgboost import XGBClassifier
from sklearn import model_selection, preprocessing
import scipy
import warnings
warnings.filterwarnings("ignore")
# Imbalance and need function

def CardAndDens(X,y):
    cardmatrix=[]
    for i in range(X.shape[0]):
        count=0
        for j in range(y.shape[1]):
            if y[i,j]==1:
                count+=1
        cardmatrix.append(count)
    Card=sum(cardmatrix)/len(cardmatrix)
    Dens=Card/y.shape[1]
    return Card,Dens
def FeatureSelect(p):
    if p==1:
        return X.toarray(),feature_names
    else:
        featurecount=int(X.shape[1]*p)
        Selectfeatureindex=[x[0] for x in (sorted(enumerate(X.sum(axis=0).tolist()[0]),key=lambda x: x[1],reverse=True))][:featurecount]
        Allfeatureindex=[i for i in range(X.shape[1])]
        featureindex=[i for i in Allfeatureindex if i not in Selectfeatureindex]
        new_x=np.delete(X.toarray(),featureindex,axis=1)
        new_featurename=[feature_names[i] for i in Selectfeatureindex] 
        return new_x,new_featurename
def ImR(X,y):
    Imr=[]
    for i in range(y.shape[1]):
        count0=0
        count1=0
        for j in range(y.shape[0]):
            if y[j,i]==1:
                count1+=1
            else:
                count0+=1
        if count1<=count0:
            Imr.append(count0/count1)
        else:
            Imr.append(count1/count0)
    return Imr
def Imbalance(X,y):
    countmatrix=[]
    for i in range(y.shape[1]):
        count0=0
        count1=0
        for j in range(y.shape[0]):
            if y[j,i]==1:
                count1+=1
            else:
                count0+=1
        countmatrix.append(count1)
    maxcount=max(countmatrix)
    ImbalanceRatioMatrix=[maxcount/i for i in countmatrix]
    MaxIR=max(ImbalanceRatioMatrix)
    MeanIR=sum(ImbalanceRatioMatrix)/len(ImbalanceRatioMatrix)
    return ImbalanceRatioMatrix,MeanIR,countmatrix
def Scumble(X,y):
    ImbalanceRatioMatrix,MeanIR,_=Imbalance(X,y)
    DifferenceImbalanceRatioMatrix=[i-MeanIR for i in ImbalanceRatioMatrix]
    count=0
    for i in range(y.shape[1]):
        count+=math.pow(DifferenceImbalanceRatioMatrix[i],2)/(y.shape[1]-1)
    ImbalanceRatioSigma=math.sqrt(count)
    CVIR=ImbalanceRatioSigma/MeanIR
    SumScumble=0
    Scumble_i=[]
    for i in range(y.shape[0]):
        count=0
        prod=1
        SumIRLbl=0
        for j in range(y.shape[1]):
            IRLbl=1
            if y[i,j]==1:
                IRLbl=ImbalanceRatioMatrix[j]
                SumIRLbl+=IRLbl
                prod*=IRLbl
                count+=1
        if count==0:
            Scumble_i.append(0)
        else:
            IRLbl_i=SumIRLbl/count
            Scumble_i.append(1.0-((1.0/IRLbl_i) * math.pow(prod, 1.0/count)))
    scumble=sum(Scumble_i)/X.shape[0]
    return Scumble_i,scumble,CVIR
def Labeltype(X,y):
    ImbalanceRatioMatrix,MeanIR,_=Imbalance(X,y)
    DifferenceImbalanceRatioMatrix=[i-MeanIR for i in ImbalanceRatioMatrix]
    MinLabelIndex=[]
    MajLabelIndex=[]
    count=0
    for i in (DifferenceImbalanceRatioMatrix):
        if i>0:
            MinLabelIndex.append(count)
        else:
            MajLabelIndex.append(count)
        count+=1
    MinLabelName=[]
    MajLabelName=[]
    for i in MinLabelIndex:
        MinLabelName.append(label_names[i][0])
    for i in MajLabelIndex:
        MajLabelName.append(label_names[i][0])
    MinLabeldic=dict(zip(MinLabelIndex,MinLabelName))
    MajLabeldic=dict(zip(MajLabelIndex,MajLabelName))
    return MinLabeldic,MajLabeldic
def CalcuNN(df1,n_neighbor):
    nbs=NearestNeighbors(n_neighbors=n_neighbor,metric='euclidean',algorithm='kd_tree').fit(df1)
    euclidean,indices= nbs.kneighbors(df1)
    return euclidean,indices
def FeatureSelect(p,X):
    if p==1:
        return X.toarray(),feature_names
    else:
        if feature_names[1][1]=='NUMERIC':
            featurecount=int(X.shape[1]*p)
            column_variances = np.var(X.toarray(), axis=0)
            sorted_indices = column_variances.argsort()[::-1]
            Selectfeatureindex = sorted_indices[:featurecount]
            Allfeatureindex=[i for i in range(X.shape[1])]
            featureindex=[i for i in Allfeatureindex if i not in Selectfeatureindex]
            new_x=np.delete(X.toarray(),featureindex,axis=1)
            new_featurename=[feature_names[i] for i in Selectfeatureindex]          
        else:
            featurecount=int(X.shape[1]*p)
            Selectfeatureindex=[x[0] for x in (sorted(enumerate(X.sum(axis=0).tolist()[0]),key=lambda x: x[1],reverse=True))][:featurecount]
            Allfeatureindex=[i for i in range(X.shape[1])]
            featureindex=[i for i in Allfeatureindex if i not in Selectfeatureindex]
            new_x=np.delete(X.toarray(),featureindex,axis=1)
            new_featurename=[feature_names[i] for i in Selectfeatureindex] 
        return new_x,new_featurename
def LabelSelect(y):
    b=[]
    new_labelname=[i for i in label_names]
    for i in range(y.shape[1]):
        if y[:,i].sum()<=20:
            b.append(i)
            new_labelname.remove(label_names[i])
    new_y=np.delete(y.toarray(),b,axis=1)
    return new_y,new_labelname 

# GET W

In [8]:
# Caculate W
def GetW(X, Y, optmParameter):
    alpha = optmParameter['alpha']
    beta = optmParameter['beta']
    gamma = optmParameter['gamma']
    lamda = optmParameter['lamda']
    lamda2 = optmParameter['lamda2']
    maxIter = optmParameter['maxIter']
    miniLossMargin = optmParameter['minimumLossMargin']
    
    # initialization
    num_dim = X.shape[1]
    XT = X.T
    XTX = np.dot(XT, X)
    XTY = np.dot(XT, Y)
    W_s = np.linalg.solve(XTX + gamma*np.eye(num_dim), XTY)
    W_s_1 = W_s
    
    # label correlation
    R = pdist(Y.T+np.finfo(float).eps, metric='cosine')
    R = 1 - squareform(R)
    C = R.reshape(Y.shape[1], Y.shape[1])
    L1 = np.diag(np.sum(C, axis=1)) - C
    
    S = ins_similarity(X, 5)
    L2 = np.diag(np.sum(S, axis=1)) - S
    iter = 1
    oldloss = 0
    bk = 1
    bk_1 = 1
    
    # compute LIP
    A = gradL21(W_s)
    Lip = np.sqrt(4*(np.linalg.norm(XTX)**2 + np.linalg.norm(alpha*XTX)**2 * np.linalg.norm(L1)**2))   
    # proximal gradient
    while iter <= maxIter:
        A = gradL21(W_s)
        W_s_k = W_s + (bk_1 - 1)/bk * (W_s - W_s_1)
        gradF = np.dot(XTX, W_s_k) - XTY + alpha * np.dot(XTX, W_s_k).dot(L1) 
        Gw_s_k = W_s_k - 1/Lip *(gradF)
        # update b, W
        bk_1 = bk
        bk = (1 + np.sqrt(4*bk**2 + 1))/2
        W_s_1 = W_s
        W_s = softthres(Gw_s_k, beta/Lip)  
        # compute loss function
        predictionLoss = np.trace(np.dot(X.dot(W_s) - Y, (X.dot(W_s) - Y).T))
        F = X.dot(W_s)
        correlation = np.trace(F.dot(L1).dot(F.T))
        In_correlation = np.trace(F.T.dot(L2).dot(F))
        sparsity = np.sum(W_s != 0)
        sparsity2 = np.trace(np.dot(W_s.T, A).dot(W_s))
        totalloss = predictionLoss + alpha*correlation + beta*sparsity 
         #  totalloss = predictionLoss + alpha*correlation + beta*sparsity + lamda*In_correlation + lamda2*sparsity2
        if abs(oldloss - totalloss) <= miniLossMargin or totalloss <= 0:
            break
        else:
            oldloss = totalloss
        iter += 1
    
    W = W_s
    return W
def softthres(W_t, lambda_):
    return np.maximum(W_t-lambda_, 0) - np.maximum(-W_t-lambda_, 0)
def ins_similarity(X, K):
    A = squareform(pdist(X))
    num_dim = A.shape[0]
    for i in range(num_dim):
        temp = A[i,:]
        As = np.sort(temp)
        temp = (temp <= As[K])
        A[i,:] = temp
    return A
def label_similarity(Y,k):
    m = Y.shape[1]
    cos_sim = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            if i == j:
                cos_sim[i, j] = 1.0
            else:
                cos_sim[i, j] = np.dot(Y[:, i], Y[:, j]) / (np.linalg.norm(Y[:, i]) * np.linalg.norm(Y[:, j]))
    upper_triangle_indices = np.triu_indices(cos_sim.shape[1], k=1)
    upper_triangle_values = cos_sim [upper_triangle_indices]
    top_k_indices = np.argpartition(upper_triangle_values, -k)[-k:]
    result_matrix = np.zeros_like(cos_sim)
    result_matrix[upper_triangle_indices[0][top_k_indices], upper_triangle_indices[1][top_k_indices]] = upper_triangle_values[top_k_indices]
    return result_matrix
def gradL21(W):
    num = W.shape[0]
    D = np.zeros((num, num))
    for i in range(num):
        temp = np.linalg.norm(W[i,:], 2)
        if temp != 0:
            D[i,i] = 1/temp
        else:
            D[i,i] = 0
    return D

# MLC METRIC

In [3]:
def macro_averaging_auc(Y, P, O):
    n = (Y.shape[0] + O.shape[0]) // 2
    l = (Y.shape[1] + O.shape[1]) // 2

    p = np.zeros(l)
    q = np.sum(Y, 0)

    zero_column_count = np.sum(q == 0)
#     print(f"all zero for label: {zero_column_count}")
    r, c = np.nonzero(Y)
    for i, j in zip(r, c):
        p[j] += np.sum((Y[ : , j] < 0.5) * (O[ : , j] <= O[i, j]))

    i = (q > 0) * (q < n)

    return np.sum(p[i] / (q[i] * (n - q[i]))) / l
def hamming_loss(Y, P, O):
    n = (Y.shape[0] + P.shape[0]) // 2
    l = (Y.shape[1] + P.shape[1]) // 2

    s1 = np.sum(Y, 1)
    s2 = np.sum(P, 1)
    ss = np.sum(Y * P, 1)

    return np.sum(s1 + s2 - 2 * ss) / (n * l)
def one_error(Y, P, O):
    n = (Y.shape[0] + O.shape[0]) // 2

    i = np.argmax(O, 1)

    return np.sum(1 - Y[range(n), i]) / n
def ranking_loss(Y, P, O):
    n = (Y.shape[0] + O.shape[0]) // 2
    l = (Y.shape[1] + O.shape[1]) // 2

    p = np.zeros(n)
    q = np.sum(Y, 1)

    r, c = np.nonzero(Y)
    for i, j in zip(r, c): 
        p[i] += np.sum((Y[i, : ] < 0.5) * (O[i, : ] >= O[i, j]))

    i = (q > 0) * (q < l)

    return np.sum(p[i] / (q[i] * (l - q[i]))) / n

# Sampling of LSDMLO

In [6]:
def Distance(df,weights,k,p1):
    n = df.shape[0]  
    distances = np.zeros((n, n))  
    data=np.array(df)
    if p1==np.inf:
        for i in range(n):
            for j in range(n):
                distances[i, j]=np.max(weights* np.abs(data[i] - data[j]))               
    else:
        for i in range(n):
            for j in range(i, n): 
                distances[i, j] = np.power(np.sum(weights * np.abs(data[i] - data[j])**p1), 1/p1)
        distances = distances + distances.T - np.diag(distances.diagonal())
    sorted_indices = np.argsort(distances, axis=1)
    sorted_indices = sorted_indices[:, ::1]
    topk_indices= sorted_indices[:, :k]
    return topk_indices
def assign_weights(arr):
    arr =np.abs(arr)
    weights = np.exp(arr - np.max(arr)) / np.sum(np.exp(arr - np.max(arr)))
    return weights
def label_assign_distance(v1, v2, w):
    distance = np.power(np.sum(w * np.abs(v1 - v2)**2), 1/2)
    return distance
def LSDMLO(df1, df2, W, sp):
    ImrMatrix=ImR(X,y)
    n_neighbors = 5
    p=2
    cos_sim = label_similarity(np.array(df2),10)
    non_zero_indices = [np.where(row != 0)[0].tolist() for row in cos_sim]
    row_sums = cos_sim.sum(axis=1)
    normalized_label_weight = cos_sim / row_sums[:, np.newaxis]
    MinLabeldic, MajLabeldic = Labeltype(np.array(df1), np.array(df2))
    ImbalanceRatioMatrix, MeanIR, countmatrix = Imbalance(np.array(df1), np.array(df2))
    MinLabelindex = list(MinLabeldic.keys())
    C = np.zeros((df1.shape[0], len(MinLabelindex)))
    C_hat=np.zeros((df1.shape[0], len(MinLabelindex))) 
    indices_dict = {}
    for tail_label in MinLabelindex:
        all_relevant=non_zero_indices[tail_label]
        sub_index = np.where(df2[MinLabeldic[tail_label]] == 1)[0]
        idx = MinLabelindex.index(tail_label)      
        W_tail_label = W[:, tail_label]
        sorted_indices = np.argsort(W_tail_label)
        sorted_column = W_tail_label[sorted_indices]
        featureWeight = assign_weights(W_tail_label)
        indices = Distance(df1, featureWeight, n_neighbors + 1, p)
        indices_dict[tail_label] = indices
        for i in range(df1.shape[0]):
            if df2.iloc[i,tail_label]==0:
                continue
            count=0
            for j in indices[i, 1:]:
                if df2.iloc[i,tail_label]==df2.iloc[j,tail_label]:
                    count +=1
            C[i, MinLabelindex.index(tail_label)] = count / n_neighbors
            count1list=[]
            if all_relevant:   
                for k in all_relevant:
                    count1=0
                    for j in indices[i, :]:
                        if df2.iloc[j, k] == 1:
                            count1+= 1
                    count1list.append(count1)
                C_hat[i, MinLabelindex.index(tail_label)]=max(count1list) / n_neighbors      
    Ins_Weight=np.zeros(df1.shape[0])  
    tem = np.zeros([df1.shape[0], len(MinLabelindex)])
    tem_hat = np.zeros([df1.shape[0], len(MinLabelindex)])
    for j in range(len(MinLabelindex)):
        SumC = 0.0
        sum_C_1 = 0.0
        c = 0
        c_1 = 0

        for i in range(df1.shape[0]):
            if C[i, j] < 1 and C[i, j] != 0:
                SumC += C[i, j]
                c += 1
            if C_hat[i, j] != 0:
                sum_C_1 += C_hat[i, j]
                c_1 += 1

            if SumC != 0.0 and c != 0:
                if C[i, j] < 1 and C[i, j] != 0:
                    tem[i, j] = C[i, j] / SumC
            else:
                tem[i, j] = 0

            if sum_C_1 != 0.0 and c_1 != 0:
                if C_hat[i, j] != 0:
                    tem_hat[i, j] = C_hat[i, j] / sum_C_1
            else:
                tem_hat[i, j] = 0
    SumW = 0
    for i in range(df1.shape[0]):
        for j in range(len(MinLabelindex)):
            if tem[i, j] != 0:
                Ins_Weight[i] += tem[i, j] + tem_hat[i, j]
        SumW += Ins_Weight[i]
    non_zero_elements = []

    for row in tem:
        for element in row:
            if element != 0:
                non_zero_elements.append(element)
                
           
    n_sample = int(df1.shape[0] * sp)
    new_X = np.zeros((n_sample, df1.shape[1]))
    target = np.zeros((n_sample, df2.shape[1]))
    count = 0 
    
    while count < n_sample:
        random_count=np.random.random()*SumW
        seed=0
        s=0
        for k in range(len(Ins_Weight)):
            s+=Ins_Weight[k]
            if(random_count<=s):
                seed=k
                break    
        seedtype = np.where(np.array(df2.iloc[seed]) == 1)[0]
        set1 = set(seedtype)
        set2 = set(MinLabelindex)
        intersection = set1.intersection(set2)
        intersection_list = list(intersection)
        if not intersection_list:
            continue  # 当 intersection_list 为空时，跳过当前循环
        select_index=np.random.choice(intersection_list)
        reference = np.random.choice(indices_dict[select_index][seed, 1:])
        all_point = indices_dict[select_index][seed, :]
        nn_df = df2[df2.index.isin(all_point)]
        ser = nn_df.sum(axis = 0, skipna = True)   
        for j in range(df1.shape[1]):
            ratio = np.random.random()
            if feature_names[j][1] == 'NUMERIC':
                new_X[count, j] = df1.iloc[seed, j] + ratio * (df1.iloc[reference, j] - df1.iloc[seed, j])
            elif feature_names[j][1] == ['YES', 'NO'] or feature_names[j][1] == ['0', '1']:
                rmd = np.random.choice([True, False])
                if rmd:
                    new_X[count, j] = df1.iloc[seed, j]
                else:
                    new_X[count, j] = df1.iloc[reference, j]
            else:
                new_X[count, j] = df1.iloc[seed, j]    
        for j in range(df2.shape[1]):
            if df2.iloc[seed, j] == df2.iloc[reference, j]:
                target[count, j]=df2.iloc[seed, j]
            else:
                featureWeight = assign_weights(W[:, j])
                distance1 = label_assign_distance(np.array(df1.iloc[seed, :]), new_X[count, :], featureWeight)
                distance2 = label_assign_distance(np.array(df1.iloc[reference, :]), new_X[count, :], featureWeight)
                if distance1 <= distance2:
                    target[count, j] = df2.iloc[seed, j]
                else:
                    target[count, j] = df2.iloc[reference, j]

        count += 1

    new_X = pd.DataFrame(new_X, columns=[x[0] for x in feature_names])
    target = pd.DataFrame(target, columns=[y[0] for y in label_names])
    LSDMLO_new_X = pd.concat([df1, new_X], axis=0).reset_index(drop=True)
    LSDMLO_target = pd.concat([df2, target], axis=0).reset_index(drop=True)
    return LSDMLO_new_X,LSDMLO_target

# Training

In [21]:
# traning
# Only the call to MLkNN is provided here. You can add other methods and control them through c_idx
def training(c_idx,sp,optmParameter):
    Randomlist=[7,20,50,21,72]
    Macro_F=[]
    Micro_F=[]
    Hamming_loss=[]
    Ranking_loss=[]
    One_error=[]
    Macro_AUC=[]
# 2-FOLD
    for i in Randomlist:  
        k_fold = IterativeStratification(n_splits=2,order=1,random_state=42)
        j=0
        for train,test in k_fold.split(X,y):
            classifier =MLkNN(k=10)
            dfx=pd.DataFrame(X[train],columns=[x[0] for x in feature_names])
            dfy=pd.DataFrame(y[train],columns=[x[0] for x in label_names])
            X1, y1 = X[train], y[train]
            W = GetW(X1, y1, optmParameter)  # 修正后的代码行
            new_X, new_y = LSDMLO(dfx, dfy, W, sp)
            X1,y1=np.array(new_X),np.array(new_y)
            classifier.fit(X1,y1)
            X2,y2=np.array(X[test]),np.array(y[test])
            ypred = classifier.predict(X2)
            if scipy.sparse.issparse(ypred):
                ypred = ypred.toarray()
            yprob = classifier.predict_proba(X2)
            if scipy.sparse.issparse(yprob):
                yprob = yprob.toarray()
            Macro_F.append(metrics.f1_score(y2, ypred,average='macro'))
            Micro_F.append(metrics.f1_score(y2, ypred,average='micro'))
            Ranking_loss.append(ranking_loss(y2, ypred, yprob))                     
            Macro_AUC.append(macro_averaging_auc(y2, ypred, yprob))  
            Hamming_loss.append(metrics.hamming_loss(y2, ypred)) 
            One_error.append(one_error(y2, ypred, yprob))

    means = np.array([
    np.mean(Macro_F),
    np.mean(Micro_F),
    np.mean(Macro_AUC),
    np.mean(Ranking_loss),
    np.mean(Hamming_loss),
    np.mean(One_error)
    ])

    stds = np.array([
        np.std(Macro_F),
        np.std(Micro_F),
        np.std(Macro_AUC),
        np.std(Ranking_loss),
        np.std(Hamming_loss),
        np.std(One_error)
    ])
    rounded_means = np.round(means, 4)
    rounded_stds = np.round(stds, 4)
    print(tuple(rounded_means) + tuple(rounded_stds))

# Main

In [22]:
# 'alpha'=lamda1,'beta'=lamda2,not give all parameter combinations
paracombine=[[2**(-2),2**(-6)],[2**(-2),2**(-4)],[2**(-1),2**(-3)],[2**(0),2**(-4)],[2**(0),2**(-2)],[2**(1),2**(-2)],[2**(2),2**(0)],[2**(3),2**(0)],[2**(3),2**(1)],[2**(4),2**(0)],[2**(4),2**(1)]]

path_to_arff_files = ["emotions.arff","rcv1subset1.arff","Corel5k.arff","yahoo-Business1","yahoo-Arts1.arff"]
label_counts = [6,101,374,28,25]
select_feature=[1,0.01,1,0.01,0.01]
data_specific_sampling=[0.01,0.02,0.05,0.1,0.15,0.3,0.5,0.7]
for i, path_to_arff_file in enumerate(path_to_arff_files):
    X, y, feature_names, label_names = load_from_arff(
        path_to_arff_file,
        label_count=label_counts[i],
        label_location="end",
        load_sparse=False,
        return_attribute_definitions=True
    )
    X,feature_names=FeatureSelect(select_feature[i],X)
    y,label_names=LabelSelect(y)
    dataset_name = path_to_arff_file.split(".")[0] 
    for c_idx in range(1,2):
        results_list=[]
        print (f"classifierid: {c_idx}")
        MacroF_list = []
        MacroAUCROC_list = []
        MacroAUCPR_list = []
        RankLoss_list = []
        res2_list = []
        res4_list = []
        res6_list = []
        res3_list = []
        for j in paracombine:
            optmParameter = {
                            'alpha':j[0], 
                            'beta': j[1],  
                            'gamma': 0.001,  
                            'lamda': 2**(1),  
                            'lamda2': 2**(5), 
                            'maxIter': 100, 
                            'minimumLossMargin': 0.01, 
                            'bQuiet': 1
                            }         
            for sp in data_specific_sampling:
                print (f"'alpha': {j[0]},'beta': {j[1]},'spratio':{sp}")
                training(c_idx,sp,optmParameter)
print("----------------------------------------------------------------------------------")

classifierid: 1
'alpha': 1,'beta': 0.0625,'spratio':0.01
(0.4357, 0.4935, 0.7833, 0.2663, 0.2697, 0.3821, 0.0196, 0.015, 0.0053, 0.0038, 0.0094, 0.0075)
'alpha': 1,'beta': 0.0625,'spratio':0.02
(0.435, 0.4937, 0.7821, 0.2662, 0.2695, 0.3872, 0.0211, 0.0135, 0.0052, 0.0045, 0.0087, 0.0099)
'alpha': 1,'beta': 0.0625,'spratio':0.05
(0.44, 0.494, 0.7805, 0.2688, 0.2695, 0.3905, 0.0203, 0.0169, 0.0059, 0.0042, 0.0062, 0.0066)
'alpha': 1,'beta': 0.0625,'spratio':0.1
(0.4537, 0.4989, 0.7779, 0.2682, 0.2747, 0.3933, 0.0341, 0.0304, 0.006, 0.0063, 0.0094, 0.0146)
'alpha': 1,'beta': 0.0625,'spratio':0.15
(0.4565, 0.5027, 0.7737, 0.2719, 0.2795, 0.4071, 0.0229, 0.0198, 0.0074, 0.0082, 0.0099, 0.0117)
'alpha': 1,'beta': 0.0625,'spratio':0.3
(0.4536, 0.4971, 0.7699, 0.2722, 0.2805, 0.4125, 0.0265, 0.0198, 0.0082, 0.0057, 0.0101, 0.0199)
'alpha': 1,'beta': 0.0625,'spratio':0.5


KeyboardInterrupt: 